In [1]:
import pandas as pd
import numpy as np
import statsmodels.tsa.filters.filtertools
import statsmodels.api as sm
import math
import re
import glob
import os
import sys
module_path = os.path.abspath(os.path.join('../../src/utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scipy.optimize import differential_evolution, NonlinearConstraint
from scipy.optimize import minimize
from datetime import datetime

from joint_optimizer import Optimizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.core.display import display, HTML
display(HTML ('<style>.container {width: 80% !important;}</style>'))
pd.options.display.float_format = '{:20,.5f}'.format

In [3]:
# time usage: 
# 1. about 6 mins for one weight_arr for global optimization, e.g. [0.99, 0.01] 
# 2. within 1 min for one weight_arr non-global optimization, e.g. [0.99, 0.01] 

# Part 1. Data Preparation

## 1). configurations: update the parameters at the below cell

In [4]:
# the modeling_stack file path used for the modelling
input_file_path = '../../data/processed_input_data/02_modelling_stack_data_seasonality.csv'
# the 'model_outputs.xlsx' file path from the model_results for sales unit, and new customer models
final_model_path_1 = '../../models/manual/sales_unit/mix_media_metrics/2021-11-17_model_outputs_v6/model_results_0802/model_outputs.xlsx'
final_model_path_2 = '../../models/manual/new_customer/mix_media_metrics/2021-11-17_model_outputs_v3/model_results_0802/model_outputs.xlsx'

# Specify whether to use the forward optimization
# if forward_opt = False, the stack dataframe from 'input_file_path' will be used for the joint optimization
# if forward_opt = True, the combined simulated dataframe from the two models will be used
# 1. if you already ran the simulator with the the above 'input_file_path', e.g. final_model_path_2, the simulated dataframe will be saved as below:
#    e.g. '../../models/manual/new_customer/mix_media_metrics/2021-08-02_model_outputs_v1/simulation_results/Updated_Stack_df.csv'
# 2. do not rename the simulated dataframe 'updated_stack_df.csv', otherwise the system cannot find it.
forward_opt = False


# Specify the weight of simulated values from each model. 
# take the variable M_ON_DIS_AT_IMP as the example, (0.5, 0.5) indicates that df['M_ON_DIS_AT_IMP'] = df_1['M_ON_DIS_AT_IMP'] * 0.5 + df_2['M_ON_DIS_AT_IMP'] * 0.5
forward_opt_model_weight = (0.5, 0.5)
    
# Select the transformation method, c_curve or s_curve
# transformation_method = 'c_curve'
transformation_method = 's_curve'

# Specify the upper and lower constraint on each of the media variable
# media_range_dict: Dictionary with upper and lower constraint for each media driver

media_range_dict = {'M_ON_DIS_AT_SPEND' : [0.7, 1.3],
                    'M_ON_DIS_CT_ROS_SPEND' : [0.7, 1.3],
                    'M_ON_DIS_HPLO_SPEND' : [0.7, 1.3],
                    'M_ON_DIS_KW_SPEND' : [0.7, 1.3],
                    'M_SEARCH_AUTO_SPEND' : [1.0, 1.0],
                    'M_SEARCH_MAN_SPEND' : [1.0, 1.0],
                    'M_OFF_DIS_WN_SPEND' : [0.7, 1.3],
                    'M_OFF_DIS_PIN_SPEND' : [0.7, 1.3],
                    'M_NATIONAL_TV_SPEND' : [1.0, 1.0],
                   }

# media_range_dict = {}
# optimization start and end dates
# if it is for future time period optimization, the optimization time period should be the same as the simulation time period. 
opt_start_end_date = ('2020-07-01', '2021-01-31')
# modeling start and end dates, RECOMMEND to use modeling start and end dates 
modeling_start_end_date = ('2019-01-01', '2021-04-30')

# the total spend will be used for the optimization:
# e.g. 1.0 represents the same spend from model
# e.g. 1.2 represents 120% of the original spend
perc = 1.0

# Specify the granular components of media (i.e. individual variables of an aggregated variable)
# granular_media_dict_1 = {'M_ON_DIS_AT_CT_IMP': ['M_ON_DIS_AT_IMP', 'M_ON_DIS_CT_IMP']}
# granular_media_dict_2 = {'M_ON_DIS_AT_CT_IMP': ['M_ON_DIS_AT_IMP', 'M_ON_DIS_CT_IMP']}

# fill out the gradular_media_dict if there are combined variables in the model
granular_media_dict_1 = {}
granular_media_dict_2 = {}

# the total incremental from function model_obj.visualize_ROAS() with the optimization period
inc_contribution_1 = 598913 
inc_contribution_2 = 598913 # 100742
# 
# inc_contribution_1 = None 
# inc_contribution_2 = None

# the max_lift from each single optimization, they are only used for optimization when goal = True
# w1 * (max_lift_1 - opt_1) **2 + w2 * (max_lift_2 - opt_) **2 
max_lift_1 = 2.01
max_lift_2 = 1.12 

# use adstock transformation or not
apply_adstock = False

# you can leave the max_life values as None if you just run the optimization with goal = False
# max_lift_1 = None 
# max_lift_2 = None

# pre-process the data stack
df = pd.read_csv(input_file_path)
df['index'] = pd.to_datetime(df['index']) 
df.set_index("index", inplace = True, drop= False)
# Please specify your custom variables that were not part of standard data stack
# Example:
df['M_SEARCH_MAN_CLK']=df['M_SBA_CLK']+df['M_SP_KWB_CLK']
df['M_SEARCH_AUTO_CLK']=df['M_SP_AB_CLK']
df['M_NATIONAL_TV_SPEND']=1000 
df['M_SEARCH_MAN_SPEND']=df['M_SBA_SPEND']+df['M_SP_KWB_SPEND']
df['M_SEARCH_AUTO_SPEND']=df['M_SP_AB_SPEND']
df['RATE_ON_DIS_CT_ROS']=df['M_ON_DIS_CT_ROS_SPEND']/df['M_ON_DIS_CT_ROS_IMP']
df['RATE_SEARCH_AUTO_CLK']=df['M_SEARCH_AUTO_SPEND']/df['M_SEARCH_AUTO_CLK']
df['RATE_SEARCH_MAN_CLK']=df['M_SEARCH_MAN_SPEND']/df['M_SEARCH_MAN_CLK']
df['RATE_NATIONAL_TV']=df['M_NATIONAL_TV_SPEND']/df['M_NATIONAL_TV_IMP']

## 2). parameters preparation: don't need to change anything at the below cell

In [5]:
# read and pass the parameters required for the optimization
input_file_path = input_file_path
final_model_path_1 = final_model_path_1
final_model_path_2 = final_model_path_2

# the parameters whether to use forward_opt
forward_opt = forward_opt
forward_opt_model_weight = forward_opt_model_weight

transformation_method = transformation_method
# optimization start and end date
opt_start_end_date = opt_start_end_date

# modeling start and end dates
modeling_start_end_date = modeling_start_end_date

perc = perc
media_range_dict = media_range_dict

# granular media dict for each model
granular_media_dict_1 = granular_media_dict_1
granular_media_dict_2 = granular_media_dict_2

# incremental contribution for each model
inc_contribution_1 = inc_contribution_1
inc_contribution_2 = inc_contribution_2

# max lift for each model
max_lift_1 = max_lift_1
max_lift_2 = max_lift_2

#
apply_adstock = apply_adstock

# Part 2. Joint Optimization 

In [6]:
# Initiate the joint optimization Object
opt_obj = Optimizer(df = df,
                forward_opt = forward_opt,
                forward_opt_model_weight = forward_opt_model_weight,
                transformation_method = transformation_method,
                opt_start_end_date = opt_start_end_date,
                modeling_start_end_date = modeling_start_end_date,
                perc = perc,
                media_range_dict = media_range_dict,
                final_model_path_1 = final_model_path_1,
                final_model_path_2 = final_model_path_2,
                granular_media_dict_1 = granular_media_dict_1,
                granular_media_dict_2 = granular_media_dict_2,
                inc_contribution_1 = inc_contribution_1,
                inc_contribution_2 = inc_contribution_2,
                max_lift_1 = max_lift_1,
                max_lift_2 = max_lift_2,
                apply_adstock = apply_adstock,
                )

Backward optimization is in use!


Variable         Original Sum
0      M_ON_DIS_AT_SPEND        407,082.80480
1  M_ON_DIS_CT_ROS_SPEND        106,781.70155
2    M_ON_DIS_HPLO_SPEND         80,448.86500
3      M_ON_DIS_KW_SPEND         19,346.25933
4     M_OFF_DIS_WN_SPEND        284,867.35832
5    M_OFF_DIS_PIN_SPEND        150,000.00000
6    M_NATIONAL_TV_SPEND        215,000.00000
7    M_SEARCH_AUTO_SPEND        472,849.46000
8     M_SEARCH_MAN_SPEND        213,420.38000
9                  Total      1,949,796.82899

In [7]:
# Function 1. call the function to export and display the joint optimization results
weights_arr = [[1, 0],  [0.75, 0.25], [0.25, 0.75],  [0, 1]]

# about goal_opt
goal_opt = False
# 1. when goal_opt = True: optimization function =  w1 * (max_lift_1 - opt_1) **2 + w2 * (max_lift_2 - opt_2) **2 
# 2. when goal_opt = False: optimization function =  w1 * opt_1 + w2 * opt_2

# about global_opt
global_opt = True
# 1. when global_opt = True: optimization method = differential_evolution
# 2. when global_opt = False: optimization method = minimize(method = 'SLSQP')

num_cores = 6

result_contrib, result_df = opt_obj.export_and_display_jointOpt_results(weights_arr, goal_opt, global_opt, num_cores)

start at 2021-12-27 22:47:40.604167
processing weight: [1, 0]...
Completed!

processing weight: [0.75, 0.25]...
Completed!

processing weight: [0.25, 0.75]...
Completed!

processing weight: [0, 1]...
Completed!

Overall Contribution result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/notebook/modelling/jointOpt_results/jointOpt_outputs_v36/result_contrib.csv


Incremental from model1  Maximized from Solver1  \
0                   598913           668,839.82899   
1                   598913           668,596.33427   
2                   598913           587,705.75680   
3                   598913           569,139.85028   

   Maximized from Solver1 True                Opt 1           True Opt 1  \
0                668,839.81735              1.11676              1.11676   
1                668,596.32263              1.11635              1.11635   
2                582,409.42876              0.98129              0.97244   
3                576,328.09887              0.95029              0.96229   

   Incremental from model2  Maximized from Solver2  \
0                   598913           107,086.63355   
1                   598913           107,325.57962   
2                   598913           185,704.29209   
3                   598913           187,812.77885   

   Maximized from Solver2 True                Opt 2           True Opt 2  \
0                107,086.63285              0.17880              0.17880   
1                107,325.57893              0.17920              0.17920   
2                187,641.74757              0.31007              0.31330   
3                188,589.86673              0.31359              0.31489   

           Difference1         %age change1          Difference2  \
0         69,926.82899             11.67562       -491,826.36645   
1         69,683.33427             11.63497       -491,587.42038   
2        -11,207.24320             -1.87126       -413,208.70791   
3        -29,773.14972             -4.97120       -411,100.22115   

          %age change2                   w1  
0            -82.11983              1.00000  
1            -82.07994              0.75000  
2            -68.99311              0.25000  
3            -68.64106              0.00000

Detailed Contribution result saved to: /Users/vn51u3a/Desktop/ms_new/Modeling/OLS Model/auto folder_structure/notebook/modelling/jointOpt_results/jointOpt_outputs_v36/result_df.csv


Variable         Original Sum    Solver Percentage  \
0      M_ON_DIS_AT_SPEND        407,082.80480              1.29996   
1  M_ON_DIS_CT_ROS_SPEND        106,781.70155              0.70039   
2    M_ON_DIS_HPLO_SPEND         80,448.86500              0.70079   
3      M_ON_DIS_KW_SPEND         19,346.25933              0.70216   
4    M_SEARCH_AUTO_SPEND        472,849.46000              1.00000   
5     M_SEARCH_MAN_SPEND        213,420.38000              1.00000   
6     M_OFF_DIS_WN_SPEND        284,867.35832              0.70016   
7    M_OFF_DIS_PIN_SPEND        150,000.00000              1.16726   
8    M_NATIONAL_TV_SPEND        215,000.00000              1.00000   
9                  Total      1,949,796.82899                  nan   

            Solver Sum                   w1             Variable.1  \
0        529,192.81908              1.00000      M_ON_DIS_AT_SPEND   
1         74,789.14621              1.00000  M_ON_DIS_CT_ROS_SPEND   
2         56,377.37637              1.00000    M_ON_DIS_HPLO_SPEND   
3         13,584.19159              1.00000      M_ON_DIS_KW_SPEND   
4        472,849.46000              1.00000    M_SEARCH_AUTO_SPEND   
5        213,420.38000              1.00000     M_SEARCH_MAN_SPEND   
6        199,451.48856              1.00000     M_OFF_DIS_WN_SPEND   
7        175,089.29675              1.00000    M_OFF_DIS_PIN_SPEND   
8        215,000.00000              1.00000    M_NATIONAL_TV_SPEND   
9      1,949,754.15856                  nan                  Total   

        Original Sum.1  Solver Percentage.1         Solver Sum.1  \
0        407,082.80480              1.29993        529,177.57102   
1        106,781.70155              0.70102         74,856.49371   
2         80,448.86500              0.70486         56,705.49102   
3         19,346.25933              0.70566         13,651.97263   
4        472,849.46000              1.00000        472,849.46000   
5        213,420.38000              1.00000        213,420.38000   
6        284,867.35832              0.70040        199,520.79820   
7        150,000.00000              1.16362        174,543.37110   
8        215,000.00000              1.00000        215,000.00000   
9      1,949,796.82899                  nan      1,949,725.53768   

                  w1.1             Variable.2       Original Sum.2  \
0              0.75000      M_ON_DIS_AT_SPEND        407,082.80480   
1              0.75000  M_ON_DIS_CT_ROS_SPEND        106,781.70155   
2              0.75000    M_ON_DIS_HPLO_SPEND         80,448.86500   
3              0.75000      M_ON_DIS_KW_SPEND         19,346.25933   
4              0.75000    M_SEARCH_AUTO_SPEND        472,849.46000   
5              0.75000     M_SEARCH_MAN_SPEND        213,420.38000   
6              0.75000     M_OFF_DIS_WN_SPEND        284,867.35832   
7              0.75000    M_OFF_DIS_PIN_SPEND        150,000.00000   
8              0.75000    M_NATIONAL_TV_SPEND        215,000.00000   
9                  nan                  Total      1,949,796.82899   

   Solver Percentage.2         Solver Sum.2                 w1.2  \
0              0.79783        324,782.61883              0.25000   
1              0.70800         75,601.23206              0.25000   
2              1.17995         94,925.77790              0.25000   
3              0.95598         18,494.61893              0.25000   
4              1.00000        472,849.46000              0.25000   
5              1.00000        213,420.38000              0.25000   
6              1.29946        370,173.32034              0.25000   
7              1.08259        162,387.87206              0.25000   
8              1.00000        215,000.00000              0.25000   
9                  nan      1,947,635.28012                  nan   

              Variable.3       Original Sum.3  Solver Percentage.3  \
0      M_ON_DIS_AT_SPEND        407,082.80480              0.79754   
1  M_ON_DIS_CT_ROS_SPEND        106,781.70155              0.7799

Total running time: 5.2 mins
